In [ ]:
import os
import shutil
import subprocess

import descarteslabs as dl
from descarteslabs.catalog import Image, properties
import matplotlib.pyplot as plt
import rasterio as rs
from rasterio.merge import merge
from tqdm import tqdm

In [ ]:
model_name = 'model_v1.1.8_5-07-21'
model_file = '../models/' + model_name + '.h5'

In [ ]:
roi_file = '../data/boundaries/nusa_tenggara.geojson'
product_id = 'Nusa_Tenggara_v1.1.8_2019-2020' 
product_desc = 'Nusa Tenggara - V1.1.8 model - 2019-2020' 
product_name = product_desc
tilesize = 840

In [ ]:
job = [
    'python3', 
    '../scripts/deploy_nn_v0.py',
    '--roi_file',
    roi_file,
    '--product_id',
    product_id,
    '--product_desc',
    product_desc,
    '--product_name',
    product_name, 
    '--model_file',
    model_file,
    '--model_name',
    model_name,
    '--tilesize',
    str(tilesize),
    '--create_product']

Launch Descartes job. Monitor at https://monitor.descarteslabs.com/

In [ ]:
output = subprocess.run(job, capture_output=True)
print(output.stderr.decode())
print(output.stdout.decode())

After the job is complete (only signaled by looking at the console), use this script to download the files. Because Descartes throws a 502 error when trying to download too many tiles. This process downloads each file individually, and then stitches them into a single geotiff

In [ ]:
search = Image.search().filter(properties.product_id == 'earthrise:' + product_id)
search.summary()

In [ ]:
output_dir = os.path.join('../data/model_outputs/heatmaps', product_id)
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

In [ ]:
image_list = [image.id for image in search]
raster_client = dl.Raster()
for image in tqdm(image_list):
    _ = raster_client.raster(inputs = image,
                             bands = ['median'],
                             save=True,
                             outfile_basename = os.path.join(output_dir, image),
                             srs='WGS84'
                        )

In [ ]:
files_to_mosaic = []
for file in os.listdir(output_dir):
    src = rs.open(os.path.join(output_dir, file))
    files_to_mosaic.append(src)
mosaic, out_trans = merge(files_to_mosaic)

In [ ]:
output_metadata = src.meta.copy()

output_metadata.update({"height": mosaic.shape[1],
                        "width": mosaic.shape[2],
                        "transform": out_trans
                 }
                )
output_metadata
with rs.open('../data/model_outputs/heatmaps/' + product_id + '.tif', 'w', **output_metadata) as f:
    f.write(mosaic)

In [ ]:
# Delete individual files and folder
shutil.rmtree(output_dir)